### Задача 0: Подключение и работа с SQLite (0б)


1. Создайте базу данных `school.db` и создайте таблицу `students` со следующими полями:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `name` (TEXT),
   - `grade` (INTEGER).
2. Вставьте данные о трёх студентах:
   - `Alice`, оценка: 85;
   - `Bob`, оценка: 90;
   - `Charlie`, оценка: 75.
3. Выведите на экран всех студентов с оценкой выше 80.
4. Обновите оценку студента `Charlie` на 80.
5. Удалите студента `Bob`.
6. Выведите всю информацию из базы данных.


In [ ]:
import sqlite3

# Подключение к базе данных
connection = sqlite3.connect("school.db")
cursor = connection.cursor()

# Создание таблицы
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    grade INTEGER NOT NULL
)
""")

# Вставка данных
cursor.executemany("INSERT INTO students (name, grade) VALUES (?, ?)", [
    ("Alice", 85),
    ("Bob", 90),
    ("Charlie", 75)
])
connection.commit()

# Вывод студентов с оценкой выше 80
cursor.execute("SELECT * FROM students WHERE grade > 80")
print("Студенты с оценкой выше 80:")
for row in cursor.fetchall():
    print(row)

# Обновление оценки Charlie
cursor.execute("UPDATE students SET grade = 80 WHERE name = 'Charlie'")
connection.commit()

# Удаление Bob
cursor.execute("DELETE FROM students WHERE name = 'Bob'")
connection.commit()

# Проверка оставшихся студентов
cursor.execute("SELECT * FROM students")
print("Оставшиеся студенты:")
for row in cursor.fetchall():
    print(row)

connection.close()

Студенты с оценкой выше 80:
(1, 'Alice', 85)
(2, 'Bob', 90)
Оставшиеся студенты:
(1, 'Alice', 85)
(3, 'Charlie', 80)


### Задача 3: Использование индексов (0.5 б)

1. Создайте SQLite-базу данных `large_library.db` с таблицей `books`:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `title` (TEXT),
   - `author` (TEXT),
   - `year` (INTEGER).
2. Вставьте 1 миллион случайных записей с помощью Python.
3. Создайте индекс на колонке `author`.
4. Напишите запрос для поиска всех книг автора `"Author_100"`. Замерьте время выполнения запроса до и после создания индекса.

In [ ]:
import sqlite3
import time
import random

# Подключение к базе данных
connection = sqlite3.connect("large_library.db")
cursor = connection.cursor()

# Создание таблицы
cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year INTEGER NOT NULL
)
""")

# Вставка данных
if not cursor.execute("SELECT COUNT(*) FROM books").fetchone()[0]:
    books = [
        (f"Book_{i}", f"Author_{random.randint(1, 1000)}", random.randint(1900, 2023))
        for i in range(1, 1000001)
    ]
    cursor.executemany("INSERT INTO books (title, author, year) VALUES (?, ?, ?)", books)
    connection.commit()

# Замер времени до индекса
start = time.time()
cursor.execute("SELECT * FROM books WHERE author = 'Author_100'")
cursor.fetchall()
print(f"Время выполнения без индекса: {time.time() - start:.4f} секунд")

# Создание индекса
cursor.execute("CREATE INDEX idx_author ON books (author)")
connection.commit()

# Замер времени после индекса
start = time.time()
cursor.execute("SELECT * FROM books WHERE author = 'Author_100'")
cursor.fetchall()
print(f"Время выполнения с индексом: {time.time() - start:.4f} секунд")

connection.close()

Время выполнения без индекса: 0.0928 секунд
Время выполнения с индексом: 0.0038 секунд


### Задача 4: Использование ограничений (constraints)  (0.5 б)

1. Создайте базу данных `university.db` с таблицей `students`:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `name` (TEXT, NOT NULL),
   - `email` (TEXT, UNIQUE),
   - `age` (INTEGER, CHECK(age >= 18)).
2. Попробуйте вставить записи:
   - `Alice`, `alice@example.com`, 20;
   - `Bob`, `bob@example.com`, 17 (должна вызвать ошибку CHECK);
   - `Charlie`, `alice@example.com`, 22 (должна вызвать ошибку UNIQUE).
3. Добавьте индекс на поле `name` для ускорения поиска.
4. Напишите запрос для выборки студентов, чей возраст больше 19.

In [ ]:
import sqlite3

# Подключение к базе данных
connection = sqlite3.connect("university.db")
cursor = connection.cursor()

# Создание таблицы с ограничениями
cursor.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    age INTEGER CHECK(age >= 18)
)
""")

# Вставка данных
try:
    cursor.executemany("INSERT INTO students (name, email, age) VALUES (?, ?, ?)", [
        ("Alice", "alice@example.com", 20),
        ("Bob", "bob@example.com", 17),  # Ошибка CHECK
        ("Charlie", "alice@example.com", 22)  # Ошибка UNIQUE
    ])
except sqlite3.IntegrityError as e:
    print(f"Ошибка вставки: {e}")
connection.commit()

# Создание индекса
cursor.execute("CREATE INDEX IF NOT EXISTS idx_name ON students (name)")
connection.commit()

# Запрос студентов старше 19 лет
cursor.execute("SELECT * FROM students WHERE age > 19")
print("Студенты старше 19 лет:")
for row in cursor.fetchall():
    print(row)

connection.close()

Ошибка вставки: UNIQUE constraint failed: students.email
Студенты старше 19 лет:
(1, 'Alice', 'alice@example.com', 20)


### Задача 2025: Создание базовой ORM-модели и работа с данными (0 б)


1. Создайте базу данных `company.db` с таблицами:
   - `departments`:
     - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
     - `name` (TEXT, UNIQUE, NOT NULL).
   - `employees`:
     - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
     - `name` (TEXT, NOT NULL),
     - `salary` (INTEGER, CHECK(salary > 0)),
     - `department_id` (INTEGER, ForeignKey(`departments.id`)).
2. Добавьте 3 отдела:
   - `HR`, `IT`, `Sales`.
3. Добавьте по 2 сотрудника в каждый отдел.
4. Напишите запросы:
   - Вывести всех сотрудников с их отделами.
   - Увеличить зарплату сотрудников отдела `IT` на 10%.
   - Удалить сотрудников из отдела `Sales`.


In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, CheckConstraint
from sqlalchemy.orm import declarative_base, sessionmaker, relationship

Base = declarative_base()

# Модели
class Department(Base):
    __tablename__ = 'departments'
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    employees = relationship("Employee", back_populates="department")

class Employee(Base):
    __tablename__ = 'employees'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    salary = Column(Integer, CheckConstraint("salary > 0"), nullable=False)
    department_id = Column(Integer, ForeignKey('departments.id'))
    department = relationship("Department", back_populates="employees")

# Создание базы данных
engine = create_engine("sqlite:///company.db")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Добавление данных
if not session.query(Department).first():
    hr = Department(name="HR", employees=[Employee(name="Alice", salary=5000), Employee(name="Bob", salary=4500)])
    it = Department(name="IT", employees=[Employee(name="Charlie", salary=6000), Employee(name="David", salary=7000)])
    sales = Department(name="Sales", employees=[Employee(name="Eve", salary=4000), Employee(name="Frank", salary=3000)])
    session.add_all([hr, it, sales])
    session.commit()

# Запросы
# 1. Вывести всех сотрудников с отделами
employees = session.query(Employee).all()
print("Все сотрудники с отделами:")
for employee in employees:
    print(f"{employee.name} - {employee.department.name}, зарплата: {employee.salary}")

# 2. Увеличить зарплату сотрудников IT на 10%
session.query(Employee).filter(Employee.department.has(name="IT")).update(
    {Employee.salary: Employee.salary * 1.1}, synchronize_session="fetch"
)
session.commit()

# 3. Удалить сотрудников из отдела Sales
sales_department = session.query(Department).filter_by(name="Sales").first()
session.query(Employee).filter_by(department_id=sales_department.id).delete()
session.commit()

# Проверка оставшихся сотрудников
print("Сотрудники после изменений:")
for employee in session.query(Employee).all():
    print(f"{employee.name} - {employee.department.name}, зарплата: {employee.salary}")

session.close()

Все сотрудники с отделами:
Alice - HR, зарплата: 5000
Bob - HR, зарплата: 4500
Charlie - IT, зарплата: 6000
David - IT, зарплата: 7000
Eve - Sales, зарплата: 4000
Frank - Sales, зарплата: 3000
Сотрудники после изменений:
Alice - HR, зарплата: 5000
Bob - HR, зарплата: 4500
Charlie - IT, зарплата: 6600.000000000001
David - IT, зарплата: 7700.000000000001


### Задача 6: Оптимизация запросов с использованием `joinedload` из ORM (0.5 б)

1. Создайте SQLite-базу данных `social.db` с таблицами:
   - `users`: `id` (INTEGER, PRIMARY KEY), `name` (TEXT);
   - `posts`: `id` (INTEGER, PRIMARY KEY), `title` (TEXT), `user_id` (INTEGER).
2. Заполните таблицы следующими данными:
   - Пользователи: `Alice`, `Bob`.
   - Посты: для `Alice` — `Post 1`, `Post 2`; для `Bob` — `Post 3`.
3. Напишите код, который эффективно выводит пользователей и их посты с использованием `joinedload`.

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.orm import declarative_base, sessionmaker, relationship, joinedload

Base = declarative_base()

# Определение моделей
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    posts = relationship("Post", back_populates="user")

class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    title = Column(String)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="posts")

# Создание базы данных
engine = create_engine("sqlite:///social.db")
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

# Добавление данных
if not session.query(User).first():
    alice = User(name="Alice", posts=[Post(title="Post 1"), Post(title="Post 2")])
    bob = User(name="Bob", posts=[Post(title="Post 3")])
    session.add_all([alice, bob])
    session.commit()

# Оптимизированный запрос
users = session.query(User).options(joinedload(User.posts)).all()
for user in users:
    print(f"User: {user.name}")
    for post in user.posts:
        print(f"  Post: {post.title}")

User: Alice
  Post: Post 1
  Post: Post 2
User: Bob
  Post: Post 3


In [ ]:
# ! rm -r *.db